In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import random
import functools
from functools import reduce
import math
from math import ceil

In [2]:
image_size = 28
input_num_units=28*28
num_labels = 10
output_num_units=num_labels
learning_rate=0.1
num_steps =501
num_train_samples = 10000
beta=0.01

In [3]:
# number of neurons in each layer
input_num_units = 784
hidden_num_units = 1024
output_num_units = 10

In [4]:
pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [5]:
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)#Concise 1-Hot Encoding in NUMPY
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def getSampleIndexes(population,sample_sz):
    if(sample_sz>population.shape[0]):
        return np.array(range(population.shape[0]))
    rand_indexes=random.sample(range(population.shape[0]),sample_sz)
    return population[rand_indexes,:],rand_indexes
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])
def isEqual(nparr1,nparr2):
    totDims=len(nparr1.shape)
    if np.sum([nparr1.shape[i]==nparr2.shape[i] for i in range(totDims)])!=totDims:
        return False
    eqItems=nparr1==nparr2
    if np.sum(eqItems)==reduce(lambda x,y:x*y,nparr1.shape):
        return True
    else:
        return False
def getEquiDistantIndexSplits(num_samples,batch_count):
    #num_samples=population.shape[0]
    n=int(ceil(num_samples/batch_count))
    sub_sample_cnts=[batch_count for i in range(n)]
    rem_indexes=num_samples%batch_count
    sub_sample_cnts=[sub_sample_cnts[i]+1 if i <rem_indexes else sub_sample_cnts[i] for i in range(len(sub_sample_cnts))]
    cum_subsmpl_cnts=np.cumsum(sub_sample_cnts)
    indices=[np.arange(0,cum_subsmpl_cnts[i]) if i==0 else np.arange(cum_subsmpl_cnts[i-1],cum_subsmpl_cnts[i])
            for i in range(cum_subsmpl_cnts.shape[0])]
    return indices

In [7]:
wt_h=None
wt_o=None
bias_h=None
bias_o=None
graph_random_weights=tf.Graph()
with  graph_random_weights.as_default():
    weights_hidden=tf.Variable(tf.random_normal([input_num_units, hidden_num_units]))
    weights_output=tf.Variable(tf.random_normal([hidden_num_units, output_num_units]))
    biases_hidden=tf.Variable(tf.random_normal([hidden_num_units]))
    biases_output= tf.Variable(tf.random_normal([output_num_units]))
with tf.Session(graph=graph_random_weights) as session:
    tf.global_variables_initializer().run()
    wt_h,wt_o,bias_h,bias_o=session.run([weights_hidden,weights_output,biases_hidden,biases_output])

In [8]:
graph_nn = tf.Graph()
def process_2layered_input(ip_tf,wt_hid,wt_op,bias_hid,bias_op):
    layer_1_linear_op=tf.add(tf.matmul(ip_tf,wt_hid),bias_hid)
    layer_1_non_lin_op=tf.nn.relu(layer_1_linear_op)
    layer_2_linear_op=tf.add(tf.matmul(layer_1_non_lin_op,wt_op),bias_op)
    #layer_2_non_lin_op=tf.nn.relu(layer_2_linear_op)
    #return layer_2_non_lin_op
    return layer_2_linear_op
with graph_nn.as_default():
    # define placeholders
    x = tf.placeholder(tf.float32, [None, input_num_units])
    y = tf.placeholder(tf.float32, [None, output_num_units])
    valid_dataset_tf = tf.placeholder(tf.float32, [None, input_num_units])
    test_dataset_tf = tf.placeholder(tf.float32, [None, input_num_units])
    weights_hidden=tf.Variable(wt_h)
    weights_output=tf.Variable(wt_o)
    biases_hidden=tf.Variable(bias_h)
    biases_output= tf.Variable(bias_o)
    output_layer=process_2layered_input(x,weights_hidden,weights_output,biases_hidden,biases_output)
    total_loss=tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output_layer)
    loss_fun = tf.reduce_mean(total_loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss_fun)
    train_prediction = output_layer
    valid_prediction=process_2layered_input(valid_dataset_tf,weights_hidden,weights_output,biases_hidden,
                                            biases_output)
    test_prediction=process_2layered_input(test_dataset_tf,weights_hidden,weights_output,biases_hidden,
                                            biases_output)

In [9]:
"""
Neural Network  Terminology: EPOCHs
    One Epoch :
         A set of (Forward,Backward) passes, s.t. these passes combined cover ALL Training Examples
    Batch Size :
        Number of Training examples in one Forward/Backward Pass. 
        Directly affects the memory requirement of the Network. Higher the batch size, more memory required.
    Number of Iterations :
        Iteration :  1 Forward Pass followed by 1 Backward Pass
        Network updates Weights Once & Only ONCE Every Iteration
        Each Iteration uses [Batch Size] number of examples. 
    Example: Given 500 training examples and a Batch size of 100 ~>
    `Network will require 5 Iterations to Complete 1 Epoch.
"""
""""""
TOTAL_TRAIN_EXAMPLES=train_dataset.shape[0]
NUM_EPOCHS=10
BATCH_SIZE=num_train_samples
ITER_INDEXES=getEquiDistantIndexSplits(TOTAL_TRAIN_EXAMPLES,BATCH_SIZE)
NUM_ITERS=len(ITER_INDEXES)

In [10]:
len(ITER_INDEXES),BATCH_SIZE

(20, 10000)

In [11]:
#With Epochs
def getFeedDict(train_x,train_y):
        return {x:train_x,y:train_y,valid_dataset_tf:valid_dataset,
              test_dataset_tf:test_dataset}
def validate_tf_comput_graph(session,tf_feed_dict):
    _=session.run([optimizer],feed_dict=tf_feed_dict)
    train_pred_1=session.run(train_prediction,feed_dict=tf_feed_dict)
    print(train_pred_1.shape)
    train_pred_2=session.run(train_prediction,feed_dict=tf_feed_dict)
    if isEqual(train_pred_1,train_pred_2):
        print("Computational Graph : Validation Passed!")
    else:
        print("Computational Graph : Validation Failed!")
with tf.Session(graph=graph_nn) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    validate_tf_comput_graph(session,getFeedDict(train_dataset[ITER_INDEXES[0]],train_labels[ITER_INDEXES[0]]))
    for epoch in range(NUM_EPOCHS):
        print("EPOCH #",epoch ,"  >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        for step,ITER_INDEX_SET in enumerate(ITER_INDEXES):
            train_x=train_dataset[ITER_INDEX_SET]
            train_y=train_labels[ITER_INDEX_SET]
            _,loss,train_pred=session.run([optimizer,loss_fun, train_prediction],feed_dict=getFeedDict(train_x,train_y))
            if step%19==0:
                print('Loss at step %d: %f' % (step, loss))
                valid_pred,test_pred=session.run([valid_prediction,test_prediction],feed_dict=getFeedDict(train_x,train_y))
                train_acc=accuracy(train_pred, train_y)
                valid_acc=accuracy(valid_pred, valid_labels)
                test_acc=accuracy(test_pred,test_labels)
                print("Accuracy ~~~~~~~~~~~~>",train_acc,valid_acc,test_acc )
                print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

Initialized
(10000, 10)
Computational Graph : Validation Passed!
EPOCH # 0   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 0: 381.152405
Accuracy ~~~~~~~~~~~~> 29.51 31.93 34.61
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 19: 55.995899
Accuracy ~~~~~~~~~~~~> 74.59 73.97 82.57
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
EPOCH # 1   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 0: 54.879944
Accuracy ~~~~~~~~~~~~> 75.07 73.33 82.07
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 19: 48.548775
Accuracy ~~~~~~~~~~~~> 75.4 74.33 82.68
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
EPOCH # 2   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 0: 48.463284
Accuracy ~~~~~~~~~~~~> 74.99 74.03 82.43
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 19